# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Setting up Connection to Database

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try: 
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    
""")
        
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# Part III. Data Modelling

## Table 1: Music Library by song duration
    Returns artist, song title, and song's length in the music app history based on session_id and item_in_session. 
    
<b> Composite Primary Keys:</b> (1) session_id, (2) item_in_session

<b> Clustering Columns:</b> (3) artist, (4) song, (5) length

<b>Info </b>: Queries will be filtered first based on the session_id and item_in_session  in the music app history and so assigning          those as the primary keys would speed up our queries. 

#### Data Modelling

In [31]:
# Table Creation
query = "CREATE TABLE IF NOT EXISTS music_library_by_song_duration"
query = query + "(session_id int, item_in_session int, artist text, song text, length decimal, , \
                    PRIMARY KEY(session_id,item_in_session))"

try: 
    session.execute(query)
    
except Exception as e:
    print(e)         

In [32]:
file = 'event_datafile_new.csv'

# Inputting data into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO music_library_by_song_duration (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        # Convert data to csv to appropriate data type 
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))) 

In [ ]:
# Fixing Errors
# Execute this code when fixing specific errors on a particular table. 
query = "DROP TABLE IF EXISTS music_library_by_song_duration"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Data Model Validation


In [46]:
query = "SELECT artist, song, length FROM music_library_by_song_duration WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

print("Artist - Song - Length \n")
for row in rows:
    print(row.artist, "-", row.song, "-", row.length)
    

Artist - Song - Length 

Faithless - Music Matters (Mark Knight Dub) - 495.3073


#### Expected Output
Faithless - Music Matters (Mark Knight Dub) - 495.3073 


#### Interpretation
    A song Music Matters or Mark Knight Dub by an artist Faithless was played in 496.3073 seconds based on our music history. It is logged in our data model having an id of item_in_session = 4 and session_id = 338. 

## Table 2: Music library by songs played by a user
    Returns artist name, song ordered by item_in_session, and the user's first name and last name. 

<b> Composite Primary Keys:</b> (1) user_id, (2) session_id, (3) item_in_session

<b> Clustering Columns:</b> (4) artist, (5) song, (6) firstname, (7) lastname

<b>Info:</b> Queries will be filtered first based on user_id and session_id in the music app history and so assigning those as primary keys would speed up our queries. item_in_session was included as primary key as it would order the results provided by the first two primary keys.

#### Data Modelling

In [63]:
# Table Creation
query = "CREATE TABLE IF NOT EXISTS music_library_by_user_songs"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, firstname text ,lastname text, \
        PRIMARY KEY((user_id, session_id),item_in_session))"

try: 
    session.execute(query)
except Exception as e:
    print(e)      

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
        query = "INSERT INTO music_library_by_user_songs (user_id, session_id, item_in_session,\
                    artist, song, firstname, lastname) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
# Fixing Errors
# Execute this code when fixing specific errors on a particular table. 
query = "DROP TABLE IF EXISTS music_library_by_user_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Data Model Validation

In [70]:
query = "SELECT artist, song, firstname, lastname FROM music_library_by_user_songs WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

print("Artist - ", "Song - ", "User Name", "\n")

for row in rows:
    print(row.artist, "-", row.song, "-" , row.firstname, row.lastname)


Artist -  Song -  User Name 

Down To The Bone - Keep On Keepin' On - Sylvie Cruz
Three Drives - Greece 2000 - Sylvie Cruz
Sebastien Tellier - Kilometer - Sylvie Cruz
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie Cruz


#### Expected Output
- Down To The Bone - Keep On Keepin' On - Sylvie Cruz
- Three Drives - Greece 2000 - Sylvie Cruz 
- Sebastien Tellier - Kilometer - Sylvie Cruz
- Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie Cruz

#### Interpretation
    There are a total of 4 songs played by user Sylvie Cruz based on our music history. 
    Results are ordered by item_in_session in ascending order. 

## Table 2: Music library by users who listens a particular song
    Returns the first name and the last name of a user who listened a particular song. 

<b> Composite Primary Keys:</b> (1) user_id, (2) song

<b> Clustering Columns:</b> (3) firstname, (4) lastname

<b>Info:</b> Queries will be filtered first based on user_id and songs.Each users are uniquely associated by their user_id.
The song is also used as a primary composite key since it will be used as a filter. 

#### Data Modelling

In [85]:
# Table Creation
query = "CREATE TABLE IF NOT EXISTS music_library_by_users"
query = query + "(song text, user_id int, firstname text, lastname text, PRIMARY KEY(song, user_id))"

try: 
    session.execute(query)
except Exception as e:
    print(e)         

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
        query = "INSERT INTO music_library_by_users (song, user_id, firstname, lastname) "
        query = query + "VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
                  

In [83]:
# Fixing Errors
# Execute this code when fixing specific errors on a particular table. 
query = "DROP TABLE IF EXISTS music_library_by_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Data Model Validation

In [86]:
query = " SELECT firstname, lastname FROM music_library_by_users WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

print("First Name", "Last Name", "\n")
for row in rows:
    print(row.firstname, row.lastname)

First Name Last Name 

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Expected Output
- Jacqueline Lynch
- Tegan Levine
- Sara Johnson

#### Interpretation
    Only 3 people have played the song "All Hands Against His Own". 
    They are Jacqueline Lynch, Tegan Levine, and Sara Johnson.

# Part IV. Cleaning and Closing the Session

### Dropping all tables created

In [88]:
query = "DROP TABLE IF EXISTS music_library_by_song_duration"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS music_library_by_user_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS music_library_by_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [89]:
session.shutdown()
cluster.shutdown()

# All Done! 